In [427]:
import scipy         
from scipy.linalg import expm
import scipy.sparse as sparse
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis, spin_basis_1d # bases
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [428]:
L = 5 #system size
M = 0.1 #fermion mass
N=3 #number of layers
l=1 #spin length
m_epsilon = 0
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [429]:
#construct basis
basis=spin_basis_1d(L=L,S = str(l))
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
       0.         |2 2 2 2 2>           242  
       1.         |2 2 2 2 1>           241  
       2.         |2 2 2 2 0>           240  
       3.         |2 2 2 1 2>           239  
       4.         |2 2 2 1 1>           238  
       5.         |2 2 2 1 0>           237  
       6.         |2 2 2 0 2>           236  
       7.         |2 2 2 0 1>           235  
       8.         |2 2 2 0 0>           234  
       9.         |2 2 1 2 2>           233  
      10.         |2 2 1 2 1>           232  
      11.         |2 2 1 2 0>           231  
      12.         |2 2 1 1 2>           230  
      13.         |2 2 1 1 1>           229  
      14.         |2 2 1 1 0>           228  
      15.         |2 2 1 0 2>           227  
      16.         |2 2 1 0 1>           226  
      17.         |2 2 1 0 0>           225  
      18.         |2 2 0 2 2>           224  
      19.         |2 2 0 2 1>           223  
      20.  

In [430]:
#initialisation of the Gauss laws

liste = []
liste_m = []
gauss_law_map = []
Gauss_law = []
gauss_law_matrix = []
z_comp_minf = [[-m_epsilon,0]]
z_comp_pinf = [[m_epsilon,L-1]]

for i in range(L+1):
    liste.append([[1.,i]])
    liste_m.append([[-1.,i]])
    if i == 0:
        gauss_law_map.append([
            ["z",liste[0]],
            ["I",z_comp_minf],

        ])
    
    elif (i == L) & (i%2 == 0):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",z_comp_pinf],
        ])
    
    elif (i == L) & (i%2 == 1):
        gauss_law_map.append([
            ["z",liste_m[i-1]],
            ["I",liste[i-1]],
            ["I",z_comp_pinf],

        ])        
        
    elif i%2 == 0:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],
        ])
    elif i%2 == 1:
        gauss_law_map.append([
            ["z",liste[i]],
            ["z",liste_m[i-1]],  
            ["I",liste[i]]
        ])
    Gauss_law.append(hamiltonian(gauss_law_map[i],dynamic_list=[],basis=basis,**no_checks))
for i in range(L+1):
    gauss_law_matrix.append(Gauss_law[i].tocsc())

In [431]:
#projectors on g=1 for every site
Proj = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L+1)]
for i in range(L+1):
    for j in range(basis.Ns):
        if (np.real(gauss_law_matrix[i][j,j])>=1.-1e-8) & (np.real(gauss_law_matrix[i][j,j])<=1.+1e-8):
            Proj[i][j,j] = 1.

In [432]:
A = np.zeros((2*l+1,2*l+1))
for i in range(2*l+1):
    for j in range(2*l+1):
        A[i][j] = (-l+i)**j
b = np.zeros(2*l+1)
for i in range(2*l+1):
    b[i] = (-1)**(-l+i)
solution = scipy.linalg.solve(A,b)
x = np.zeros(2*3+1)
for i in range(2*l+1):
    x[i] = solution[i]
print(x)

[ 1.  0. -2.  0.  0.  0.  0.]


In [433]:
const_term = []
linear_term = []
quadratic_term = []
cubic_term = []
quartic_term = []
quintic_term = []
sextic_term = []
interaction_p = []
interaction_m = []

for i in range(L-1):
    const_term.append([[x[0]/(2*np.sqrt(l*(l+1))),i]])
    linear_term.append([[x[1]/(2*np.sqrt(l*(l+1))),i,i+1]])
    quadratic_term.append([[x[2]/(2*np.sqrt(l*(l+1))),i,i+1,i+1]])
    cubic_term.append([[((-1)**i*x[3])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1]])
    quartic_term.append([[((-1)**i*x[4])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1]])
    quintic_term.append([[((-1)**i*x[5])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1]])
    sextic_term.append([[((-1)**i*x[6])/(2*np.sqrt(l*(l+1))),i,i+1,i+1,i+1,i+1,i+1,i+1]])

    const_term

    interaction_p.append([
        ["+", const_term[i]],
        ["+z", linear_term[i]],
        ["+zz", quadratic_term[i]],
        ["+zzz", cubic_term[i]],
        ["+zzzz", quartic_term[i]],
        ["+zzzzz", quintic_term[i]],
        ["+zzzzzz", sextic_term[i]],

    ])

    interaction_m.append([
        ["-", const_term[i]],
        ["-z", linear_term[i]],
        ["-zz", quadratic_term[i]],
        ["-zzz", cubic_term[i]],
        ["-zzzz", quartic_term[i]],
        ["-zzzzz", quintic_term[i]],
        ["-zzzzzz", sextic_term[i]],

    ])

const_term.append([[(0.5*(-1)**m_epsilon)/np.sqrt(l*(l+1)),L-1]])

interaction_p.append([
    ["+", const_term[L-1]]
])
interaction_m.append([
    ["-", const_term[L-1]]
])

kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[2*(-1)**i*M,i] for i in range(L)]

kin_mass_map = [
            ["zz", kin_energy], 
            ["z", mass_term], 
]

H_int_p = []
H_int_m = []
interaction = [sparse.csr_matrix((basis.Ns,basis.Ns)) for i in range(L)]
h_int = sparse.csr_matrix((basis.Ns,basis.Ns))
for i in range(L):
    H_int_p.append(hamiltonian(interaction_p[i],dynamic_list=[],basis=basis,**no_checks))
    H_int_m.append(hamiltonian(interaction_m[i],dynamic_list=[],basis=basis,**no_checks))
    interaction[i] = (Proj[i]@H_int_p[i].tocsc()@Proj[i+1]+Proj[i+1]@H_int_m[i].tocsc()@Proj[i])

H_kin_mass = hamiltonian(kin_mass_map,dynamic_list=[],basis=basis,**no_checks)
h_kin_mass = H_kin_mass.tocsc()

h_int = 0

for i in range(L):
    h_int += interaction[i]


full_ham_matrix = h_kin_mass + h_int #Spin-1 Hamiltonian without the local contstaints

G=0
for i in range(L+1):
    G += 1e2*(Gauss_law[i]**2-Gauss_law[i])**2
constraint_full_matrix = G.tocsc()
constrained_full_ham_matrix= full_ham_matrix + constraint_full_matrix #Spin-1 Hamiltonian with the local contstaints as a penalty term
#print(full_ham_matrix)

In [434]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian without the local contstaints
eigenval, eigenvec = scipy.sparse.linalg.eigsh(full_ham_matrix,k = 3,which = "SA")
print(eigenval[eigenval<=1e1])
print(eigenvec[:,0])

[-1.0416023  -0.34551385 -0.34551385]
[ 3.97022418e-03-1.08481717e-03j  5.35575217e-18+1.16766338e-17j
  1.67450496e-18-1.58385758e-18j  1.48549999e-02-4.05895441e-03j
 -2.77693023e-02+7.58763598e-03j -1.04821597e-17+1.70528255e-18j
 -1.94130923e-17-1.44083021e-17j  1.73089123e-18+5.02011537e-18j
 -5.96268195e-18-3.45442733e-18j -8.13434395e-18-9.58813789e-18j
 -7.04666483e-18+3.22811261e-19j -2.26454915e-18-2.52095111e-17j
  1.00643882e-17-1.70776070e-17j -5.50780417e-18-1.34519294e-17j
  1.72024659e-17-2.39261484e-18j -1.22121228e-18+5.25974071e-18j
 -3.79803909e-18+1.94993647e-17j  3.84056981e-18-3.39589583e-18j
  1.39402226e-19-4.86904638e-19j  6.34552280e-18-2.57792079e-17j
  4.58593430e-19+1.16974061e-17j  7.57216460e-18+3.26402331e-18j
 -1.29044981e-17-6.72465468e-18j  1.82597426e-17+7.66923890e-18j
  7.23732950e-18-2.29403893e-19j  8.18768192e-18-6.65444233e-19j
  7.49796571e-19+4.40954283e-18j  1.48549999e-02-4.05895441e-03j
 -2.75818509e-18+6.76276565e-18j  2.86186542e-18-1.8

In [435]:
#eigenvalues and eigenvectors of the Spin-1 Hamiltonian with the local contstaints
eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(constrained_full_ham_matrix,k = 3,which = "SA")


In [436]:
#Gauge invariant part of the spectrum
string = ""
print(eigenvalues[eigenvalues<=1e1])
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)


[-1.0416023  -0.0142565   0.15174042]
(-0.0030017899921038877-0.0028158063083915377j)|2 2 2 2 2>
(-0.01123150434911446-0.010535627369811226j)|2 2 2 1 2>
(0.020995694443849994+0.019694851744880743j)|2 2 2 1 1>
(-0.011231504349155146-0.010535627369721115j)|2 1 2 2 2>
(-0.05331125804211569-0.05000822079449204j)|2 1 2 1 2>
(0.10389218414958284+0.09745527444976455j)|2 1 2 1 1>
(0.10471816513105664+0.09823007963759711j)|2 1 1 1 2>
(-0.2290244841743082-0.21483467831253603j)|2 1 1 1 1>
(0.0863555949881831+0.0810052101488039j)|2 1 1 0 1>
(0.020995694443760312+0.019694851744973672j)|1 1 2 2 2>
(0.10389218414949343+0.09745527445003588j)|1 1 2 1 2>
(-0.2039953555962125-0.1913562943051796j)|1 1 2 1 1>
(-0.22902448417443833-0.21483467831278547j)|1 1 1 1 2>
(0.5027731940937056+0.47162257697769666j)|1 1 1 1 1>
(-0.1926675551007114-0.1807303370666639j)|1 1 1 0 1>
(0.08635559498814475+0.08100521014882706j)|1 0 1 1 2>
(-0.19266755510063407-0.18073033706632566j)|1 0 1 1 1>
(0.08197172643895172+0.076892955

In [437]:
#initial state
string = ""
for i in range(L):
    string +="1"
psi_0 = np.zeros(basis.Ns)
i_0 = basis.index(string)
psi_0[i_0] = 1.

In [438]:
#cost function of the initial state
expt_value = 0
matvec_h = full_ham_matrix@psi_0
matvec_c = constraint_full_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*(matvec_h[i]+matvec_c[i])
print(np.real(expt_value))

0.0


In [439]:
expt_value = 0
matvec = full_ham_matrix@psi_0
for i in range(basis.Ns):
    expt_value += np.conj(psi_0[i])*matvec[i]
print(expt_value)

0j


In [466]:
def rot_sigma(k,i,j,phi,theta):
    if k>=L:
        print("Warning! Rotations apply to k<=L-1!")
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    rot_matrix = expm(-1j*theta*sigma/2)
    if k==0:
        tensprod = rot_matrix
        for i in range(L-1):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+1,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms(k,i,j,phi,theta):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))
    sigma = sparse.csc_matrix(sigma)
    power_matrix = scipy.sparse.kron(sigma,sparse.identity(2*l+1))+scipy.sparse.kron(sparse.identity(2*l+1),sigma)
    rot_matrix = scipy.sparse.linalg.expm(-1j*power_matrix@power_matrix*theta/4)
    if k == 0:
        tensprod = rot_matrix
        for i in range(L-2):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    else:
        tensprod = sparse.identity(2*l+1)
        for i in range(1,k):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        tensprod = scipy.sparse.kron(tensprod,rot_matrix)
        for i in range(k+2,L):
            tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
    return tensprod

def rot_ms_e(i,j,phi,theta):
    ms_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        ms_e = rot_ms(2*k,i,j,phi,theta)@ms_e
    return(ms_e)

def rot_ms_o(i,j,phi,theta):
    ms_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)-1):
        ms_o = rot_ms(2*k+1,i,j,phi,theta)@ms_o
    return(ms_o)

def rot_sigma_e(i,j,phi,theta):
    sigma_e = sparse.identity(basis.Ns,dtype = complex)
    for k in range(1,int(L/2)):
        sigma_e = rot_sigma(2*k,i,j,phi,theta)@sigma_e
    return(sigma_e)

def rot_sigma_o(i,j,phi,theta):
    sigma_o = sparse.identity(basis.Ns,dtype = complex)
    for k in range(int(L/2)):
        sigma_o = rot_sigma(2*k+1,i,j,phi,theta)@sigma_o
    return(sigma_o)

def rot_sigma_all(i,j,phi,theta):
    sigma_all = sparse.identity(basis.Ns,dtype = complex)
    for k in range(L):
        sigma_all = rot_sigma(k,i,j,phi,theta)@sigma_all
    return(sigma_all)

In [467]:
def ms_hamiltonian(i,j,phi):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-i][2*l-j] = 0.5*(np.cos(phi)+1j*np.sin(phi))
    sigma[2*l-j][2*l-i] = 0.5*(np.cos(phi)-1j*np.sin(phi))

    sigma = sparse.csc_matrix(sigma)
    summe = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for k in range(L):
        if k == 0:
            tensprod = sigma
            for m in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        elif k == int(L/2):  
            tensprod = np.zeros((basis.Ns,basis.Ns))

        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = tensprod.toarray()
        summe +=tensprod
    return summe@summe
                
def sigma_z_part(i,j):
    sigma = np.zeros((2*l+1,2*l+1),dtype = complex)
    sigma[2*l-j][2*l-j] = -1.
    sigma = sparse.csc_matrix(sigma)
    summe = 0
    for k in range(L):
        tensprod = 0
        if k==0:
            tensprod = sigma
            for i in range(L-1):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        else:
            tensprod = sparse.identity(2*l+1)
            for i in range(1,k):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
            tensprod = scipy.sparse.kron(tensprod,sigma)
            for i in range(k+1,L):
                tensprod = scipy.sparse.kron(tensprod,sparse.identity(2*l+1))
        summe +=tensprod
    return summe
    
matrix_part = ms_hamiltonian(1,2,0)
ms_ham_eigenval, ms_ham_eigenvec = scipy.linalg.eigh(matrix_part)




def ms_gate_global(theta):
    exponential = np.zeros((basis.Ns,basis.Ns),dtype = complex)
    for i in range(basis.Ns):
        exponential[i][i] = np.exp(1j*ms_ham_eigenval[i]*theta/4)
    exponential = scipy.sparse.csc_matrix(exponential)
    return ms_ham_eigenvec@exponential@ms_ham_eigenvec.T.conj()

    

In [468]:
#import timeit
#print(timeit.timeit('[ms_gate_global(0,2,np.pi)]', globals=globals()))
import datetime
print(datetime.datetime.now())
a = ms_gate_global(np.pi)
print(datetime.datetime.now())
print(np.size(a))


2022-03-22 17:20:59.925594
2022-03-22 17:20:59.929414
59049


In [469]:
opt_params = []
function_values = []
def callback_function(x,fun,context):
    opt_params.append(x)
    function_values.append(fun)


In [470]:
full_ham_matrix = sparse.csc_matrix(full_ham_matrix)

In [471]:
N=4
theta = np.zeros(7*N)
import optuna
#cost function definition
def cost_function_sigma(trial):
    psi_var = psi_0
    for i in range(N):
        theta[7*i+0] = trial.suggest_float("7*"+str(i)+"+0", -3*np.pi, 3*np.pi)
        theta[7*i+1] = trial.suggest_float("7*"+str(i)+"+1", -3*np.pi, 3*np.pi)
        theta[7*i+2] = trial.suggest_float("7*"+str(i)+"+2", -3*np.pi, 3*np.pi)
        theta[7*i+3] = trial.suggest_float("7*"+str(i)+"+3", -3*np.pi, 3*np.pi)
        theta[7*i+4] = trial.suggest_float("7*"+str(i)+"+4", -3*np.pi, 3*np.pi)
        theta[7*i+5] = trial.suggest_float("7*"+str(i)+"+5", -3*np.pi, 3*np.pi)
        theta[7*i+6] = trial.suggest_float("7*"+str(i)+"+6", -3*np.pi, 3*np.pi)

        psi_var = ms_gate_global(theta[7*i+0])@psi_var
        psi_var = rot_sigma_e(0,2,0,theta[7*i+2])@rot_sigma_e(0,1,0,theta[7*i+1])@psi_var
        psi_var = rot_sigma_o(0,2,0,theta[7*i+4])@rot_sigma_o(0,1,0,theta[7*i+3])@psi_var
        psi_var = rot_sigma(0,0,2,0,theta[7*i+6])@rot_sigma(0,0,1,0,theta[7*i+5])@psi_var
        psi_var = rot_sigma(L-1,0,2,0,theta[7*i+6])@rot_sigma(L-1,0,1,0,theta[7*i+5])@psi_var
        
    cost = np.conj(psi_var)@full_ham_matrix@psi_var
    return np.real(cost)

if __name__ == "__main__":
    sampler = optuna.samplers.CmaEsSampler()
    study = optuna.create_study(sampler=sampler)
    study.optimize(cost_function_sigma, n_trials=3000)

[I 2022-03-22 17:21:01,353] A new study created in memory with name: no-name-eccc8402-6962-4cff-af1d-0441940c1aa3
[I 2022-03-22 17:21:01,540] Trial 0 finished with value: 1.5029195692245114 and parameters: {'7*0+0': -8.286643976973757, '7*0+1': -3.401310230377569, '7*0+2': 3.860354934416332, '7*0+3': 3.7723779202032652, '7*0+4': 5.516013304583449, '7*0+5': -7.73882318475689, '7*0+6': -4.37845781656268, '7*1+0': 6.899737017308318, '7*1+1': 7.772138296668317, '7*1+2': 3.3128261842627573, '7*1+3': 8.887918021206907, '7*1+4': -9.216127605162033, '7*1+5': -6.961855764875702, '7*1+6': 2.367798852458991, '7*2+0': 9.119531840716448, '7*2+1': -9.022449015159616, '7*2+2': 6.795093971785569, '7*2+3': -7.113595654905162, '7*2+4': -1.7665500152063336, '7*2+5': -5.258837414366717, '7*2+6': -1.1109464124360446, '7*3+0': 6.56675680911836, '7*3+1': 7.89792689280393, '7*3+2': 3.374794335050609, '7*3+3': -3.7088291030380374, '7*3+4': 9.00607919898652, '7*3+5': -4.773975323529323, '7*3+6': 5.0707354844119

[I 2022-03-22 17:21:02,958] Trial 9 finished with value: 1.4198758936472065 and parameters: {'7*0+0': -0.7006120999456169, '7*0+1': 0.13099788582357258, '7*0+2': 4.135067600846171, '7*0+3': -4.170649958812028, '7*0+4': -0.05941160560573864, '7*0+5': -2.1451175401824556, '7*0+6': 0.8259591493893774, '7*1+0': 5.146548071031041, '7*1+1': -0.2265450787859908, '7*1+2': 0.7553109083798037, '7*1+3': -3.0470762538257223, '7*1+4': -2.4224585184313874, '7*1+5': 6.9921689102413245, '7*1+6': -2.348333341792281, '7*2+0': -1.3157118627761106, '7*2+1': -1.8780280900133404, '7*2+2': 2.384992210624672, '7*2+3': 2.437209322272882, '7*2+4': -2.941883741688696, '7*2+5': 4.453179749546257, '7*2+6': 0.23532840470877942, '7*3+0': 2.5089451339986946, '7*3+1': -0.8043081370674645, '7*3+2': -3.287522469979658, '7*3+3': 3.907769242252118, '7*3+4': 2.5781110745185054, '7*3+5': 1.2632003539550838, '7*3+6': 2.007488768221718}. Best is trial 5 with value: 0.7954562798164013.
[I 2022-03-22 17:21:03,130] Trial 10 fini

[I 2022-03-22 17:21:04,570] Trial 18 finished with value: 1.641680671670466 and parameters: {'7*0+0': -1.3710700189471137, '7*0+1': -4.467482007395903, '7*0+2': 2.6290231480727138, '7*0+3': 3.9586014621718846, '7*0+4': -1.0275035406176198, '7*0+5': 1.4673420120028022, '7*0+6': 1.108535999551754, '7*1+0': -1.2326144362062261, '7*1+1': 5.162016407108366, '7*1+2': 3.7508660922710586, '7*1+3': -0.7723993846938223, '7*1+4': -2.4604497386826365, '7*1+5': 2.3104144496555414, '7*1+6': -2.8398816644690363, '7*2+0': -3.4134667658631095, '7*2+1': -3.2307704164857975, '7*2+2': 0.0032791796835284215, '7*2+3': -1.935345016006709, '7*2+4': 1.4668295824016457, '7*2+5': 6.212099980348145, '7*2+6': -2.5329483111625555, '7*3+0': -0.4863404061357011, '7*3+1': -0.3783910517645567, '7*3+2': 1.2933205691233707, '7*3+3': 2.2696820813654033, '7*3+4': 0.6612719549218147, '7*3+5': 5.106936413273362, '7*3+6': -1.6297861669477784}. Best is trial 5 with value: 0.7954562798164013.
[I 2022-03-22 17:21:04,772] Trial 1

[I 2022-03-22 17:21:06,072] Trial 27 finished with value: 1.754132509384478 and parameters: {'7*0+0': -0.8531574657584053, '7*0+1': -8.98807679268401, '7*0+2': -0.9203480898465644, '7*0+3': -2.8815894468962373, '7*0+4': -2.715613026365191, '7*0+5': -1.9226714689118598, '7*0+6': -3.622057842276568, '7*1+0': 2.169995608887268, '7*1+1': -2.6066731976742066, '7*1+2': 2.6034535969971544, '7*1+3': 1.233483749533417, '7*1+4': 0.9534544312369567, '7*1+5': -0.6132805573655374, '7*1+6': 0.6190419862801197, '7*2+0': -4.36507785932457, '7*2+1': -4.723052803233461, '7*2+2': -0.14277039839083017, '7*2+3': -0.5293520989249405, '7*2+4': -0.49057766842047484, '7*2+5': 3.0210681766963057, '7*2+6': 0.808140795595328, '7*3+0': -3.6424675685267536, '7*3+1': 5.731706763752035, '7*3+2': -3.2945061396808466, '7*3+3': -2.444037026893152, '7*3+4': 1.0105892668911718, '7*3+5': 3.7366445831443453, '7*3+6': 4.512848201415972}. Best is trial 5 with value: 0.7954562798164013.
[I 2022-03-22 17:21:06,242] Trial 28 fin

[I 2022-03-22 17:21:07,495] Trial 36 finished with value: 1.4517620584727653 and parameters: {'7*0+0': 3.1689208046471364, '7*0+1': -1.5367171761892748, '7*0+2': -0.5546141565597592, '7*0+3': 1.4485399175299134, '7*0+4': 7.129835841964497, '7*0+5': -4.4389848261254325, '7*0+6': -5.5814540088206055, '7*1+0': -1.8553942504495524, '7*1+1': 3.6916231551449274, '7*1+2': 5.595421064521716, '7*1+3': 5.176193007412211, '7*1+4': 4.615205984327055, '7*1+5': -1.3721615048842495, '7*1+6': -4.079295418258462, '7*2+0': -1.5418093818015852, '7*2+1': -2.416071814543014, '7*2+2': 1.037754082856294, '7*2+3': 1.2361166525357876, '7*2+4': 4.229411632393382, '7*2+5': 3.0639297059028707, '7*2+6': -0.707720227203391, '7*3+0': 6.283136283279025, '7*3+1': 1.546028849369249, '7*3+2': -2.4276583610213485, '7*3+3': 0.5081987852899851, '7*3+4': -3.681689271648672, '7*3+5': -7.053139690713713, '7*3+6': 5.986394545310522}. Best is trial 5 with value: 0.7954562798164013.
[I 2022-03-22 17:21:07,643] Trial 37 finished 

...

[I 2022-03-22 17:29:52,305] Trial 2961 finished with value: -0.9509408839771489 and parameters: {'7*0+0': -0.15215254466382533, '7*0+1': -2.215725932839017, '7*0+2': -7.647745085691381, '7*0+3': -1.218102707697927, '7*0+4': -5.89882530387802, '7*0+5': -3.2629280246487284, '7*0+6': -5.984339488935198, '7*1+0': -1.0593130460834668, '7*1+1': -1.8522018819723047, '7*1+2': -4.778520190511139, '7*1+3': 0.716811092779464, '7*1+4': 1.0548451792275462, '7*1+5': -7.7982880374625925, '7*1+6': 1.7753118866789575, '7*2+0': 1.2202452834905915, '7*2+1': -1.0339636053251824, '7*2+2': 4.965596817358927, '7*2+3': 0.5196428474958859, '7*2+4': -4.561422763098633, '7*2+5': 0.31265105345006133, '7*2+6': 2.904709071246496, '7*3+0': 0.1608374536674383, '7*3+1': 2.088461756783656, '7*3+2': 2.4090083457403244, '7*3+3': -1.724667193968935, '7*3+4': -2.8797382031799676, '7*3+5': -4.294609698938717, '7*3+6': 2.587028965135865}. Best is trial 2941 with value: -0.9513942505047447.
[I 2022-03-22 17:29:52,479] Trial 2

[I 2022-03-22 17:29:54,125] Trial 2970 finished with value: -0.9504913097314069 and parameters: {'7*0+0': -0.15535255193019337, '7*0+1': -2.1438015393668524, '7*0+2': -7.647329040419467, '7*0+3': -1.275505339475048, '7*0+4': -5.788098408620386, '7*0+5': -3.296077159587282, '7*0+6': -5.9335565863471995, '7*1+0': -1.1408298047387948, '7*1+1': -1.863344360671937, '7*1+2': -4.780444587281113, '7*1+3': 0.6225846279495861, '7*1+4': 0.972935569015328, '7*1+5': -7.723228913857224, '7*1+6': 1.8399296529900928, '7*2+0': 1.3171921795253856, '7*2+1': -0.9984086224681599, '7*2+2': 4.979281385770435, '7*2+3': 0.6465619091748122, '7*2+4': -4.58863297454888, '7*2+5': 0.3667389295445609, '7*2+6': 2.8296889082445067, '7*3+0': 0.24553816038778284, '7*3+1': 2.079341498100125, '7*3+2': 2.364634652862619, '7*3+3': -1.7384354711315575, '7*3+4': -2.805144907294372, '7*3+5': -4.35047659365884, '7*3+6': 2.5635459758848382}. Best is trial 2964 with value: -0.9514873106702172.
[I 2022-03-22 17:29:54,314] Trial 29

[I 2022-03-22 17:29:55,898] Trial 2979 finished with value: -0.9504849689128854 and parameters: {'7*0+0': -0.16111852233825144, '7*0+1': -2.1348597711019077, '7*0+2': -7.651016717395821, '7*0+3': -1.2508655856512263, '7*0+4': -5.900347099630041, '7*0+5': -3.296523634805312, '7*0+6': -5.928340814100995, '7*1+0': -1.1537172664687088, '7*1+1': -1.955055961162802, '7*1+2': -4.813506148999561, '7*1+3': 0.6635339603385569, '7*1+4': 1.0488335773918869, '7*1+5': -7.801271716299999, '7*1+6': 1.8266522085292898, '7*2+0': 1.1819283628337436, '7*2+1': -1.0266604964752946, '7*2+2': 4.951276700344808, '7*2+3': 0.6652725992012141, '7*2+4': -4.551197111353845, '7*2+5': 0.3402254493775851, '7*2+6': 2.859792651330356, '7*3+0': 0.1830744432413827, '7*3+1': 2.187540707887269, '7*3+2': 2.33107833594777, '7*3+3': -1.6874734044798143, '7*3+4': -2.825665833402419, '7*3+5': -4.306825766217008, '7*3+6': 2.472619934240614}. Best is trial 2964 with value: -0.9514873106702172.
[I 2022-03-22 17:29:56,085] Trial 298

[I 2022-03-22 17:29:57,618] Trial 2988 finished with value: -0.9516893388232786 and parameters: {'7*0+0': -0.12819195254951204, '7*0+1': -2.2208299915090914, '7*0+2': -7.725438177429764, '7*0+3': -1.2004216528534806, '7*0+4': -5.8606747782725295, '7*0+5': -3.257738785950246, '7*0+6': -5.893538951432685, '7*1+0': -1.106355689574329, '7*1+1': -1.913524986442395, '7*1+2': -4.896342416774316, '7*1+3': 0.7070549847609715, '7*1+4': 1.0637496414418048, '7*1+5': -7.788891617235086, '7*1+6': 1.809357150201693, '7*2+0': 1.198156997989507, '7*2+1': -1.0402185193642952, '7*2+2': 5.108978125788519, '7*2+3': 0.5719758922739808, '7*2+4': -4.551621035765104, '7*2+5': 0.35459279721918296, '7*2+6': 2.895260012752178, '7*3+0': 0.21278316600138467, '7*3+1': 2.1052953636163774, '7*3+2': 2.4096602954054873, '7*3+3': -1.7148571737917853, '7*3+4': -2.8753260304861934, '7*3+5': -4.315182462304058, '7*3+6': 2.546489708120111}. Best is trial 2988 with value: -0.9516893388232786.
[I 2022-03-22 17:29:57,793] Trial

[I 2022-03-22 17:29:59,314] Trial 2997 finished with value: -0.9520307542283046 and parameters: {'7*0+0': -0.15291237513617897, '7*0+1': -2.1711586776982705, '7*0+2': -7.654571162839915, '7*0+3': -1.1945658671837396, '7*0+4': -5.927052178918216, '7*0+5': -3.250055573298547, '7*0+6': -5.9586585359123365, '7*1+0': -1.0984507422034324, '7*1+1': -1.9037060169067963, '7*1+2': -4.86738426123815, '7*1+3': 0.6863183615401409, '7*1+4': 1.098335431811985, '7*1+5': -7.837503877969166, '7*1+6': 1.7919025416036471, '7*2+0': 1.1804514269880053, '7*2+1': -1.0167027621984095, '7*2+2': 5.039066857495932, '7*2+3': 0.5520807473450389, '7*2+4': -4.545337457322669, '7*2+5': 0.3658313426827164, '7*2+6': 2.9085780010345936, '7*3+0': 0.17417782925190664, '7*3+1': 2.1322640988036268, '7*3+2': 2.3591326749335337, '7*3+3': -1.630192141916587, '7*3+4': -2.8956464659197056, '7*3+5': -4.368401490902199, '7*3+6': 2.4588501746525386}. Best is trial 2997 with value: -0.9520307542283046.
[I 2022-03-22 17:29:59,493] Tri

In [472]:
theta = np.zeros(7*N)
i=0
for value in study.best_trial.params.items():
    theta[i] = value[1]
    i+=1

psi_var = psi_0
for i in range(N):
    psi_var = ms_gate_global(theta[7*i+0])@psi_var
    psi_var = rot_sigma_e(0,2,0,theta[7*i+2])@rot_sigma_e(0,1,0,theta[7*i+1])@psi_var
    psi_var = rot_sigma_o(0,2,0,theta[7*i+4])@rot_sigma_o(0,1,0,theta[7*i+3])@psi_var
    psi_var = rot_sigma(0,0,2,0,theta[7*i+6])@rot_sigma(0,0,1,0,theta[7*i+5])@psi_var
    psi_var = rot_sigma(L-1,0,2,0,theta[7*i+6])@rot_sigma(L-1,0,1,0,theta[7*i+5])@psi_var

print(np.abs(np.dot(np.conj(psi_var),eigenvec[:,0]))**2)

0.9556482552917559


In [475]:
string = ""
for i in range(basis.Ns):
    if np.abs(psi_var[i])>=1e-3:
        string+=str(psi_var[i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(0.0007336992105729985-0.0011353290350443698j)|2 2 2 1 1>
(0.0012215162980002746+0.0004428634307251109j)|2 2 2 0 1>
(0.000377501615418442+0.0016579057366813204j)|2 2 1 2 1>
(0.0004510962276678045-0.0025806688935718124j)|2 2 1 1 2>
(-0.0021829885064922464+0.0033779649737598564j)|2 2 1 1 1>
(0.0018476031662521953+0.001784972467870522j)|2 2 1 1 0>
(0.0016948872932911266+0.00014972642102181044j)|2 2 1 0 2>
(-0.003634399493145606-0.0013176595603313019j)|2 2 1 0 1>
(0.0007336992105727505-0.0011353290350453172j)|2 1 2 2 1>
(-0.04224318973873965+0.0245398854063827j)|2 1 2 1 2>
(0.08681368262188106-0.0642791408439648j)|2 1 2 1 1>
(-0.0005928388773700569-0.0012094882740762772j)|2 1 2 1 0>
(0.00351802773027992-0.0028116827774859513j)|2 1 2 0 2>
(-0.016036996260721866+0.015152393545772622j)|2 1 2 0 1>
(0.0004510962276674944-0.002580668893571841j)|2 1 1 2 2>
(-0.002182988506491646+0.003377964973759459j)|2 1 1 2 1>
(0.0018476031662523922+0.001784972467870199j)|2 1 1 2 0>
(0.1256869250346268-0.073013

In [474]:
string = ""
for i in range(basis.Ns):
    if np.abs(eigenvectors[:,0][i])>=1e-5:
        string+=str(eigenvectors[:,0][i])+basis.int_to_state(basis.Ns-i-1)+"\n"
        #for j in range(basis.Ns):
            #if (np.abs(np.abs(eigenvectors[:,0][i])-np.abs(eigenvectors[:,0][j]))<=1e-5) & (i<j):
                #print(basis.int_to_state(basis.Ns-i-1),basis.int_to_state(basis.Ns-j-1))
print(string)

(-0.0030017899921038877-0.0028158063083915377j)|2 2 2 2 2>
(-0.01123150434911446-0.010535627369811226j)|2 2 2 1 2>
(0.020995694443849994+0.019694851744880743j)|2 2 2 1 1>
(-0.011231504349155146-0.010535627369721115j)|2 1 2 2 2>
(-0.05331125804211569-0.05000822079449204j)|2 1 2 1 2>
(0.10389218414958284+0.09745527444976455j)|2 1 2 1 1>
(0.10471816513105664+0.09823007963759711j)|2 1 1 1 2>
(-0.2290244841743082-0.21483467831253603j)|2 1 1 1 1>
(0.0863555949881831+0.0810052101488039j)|2 1 1 0 1>
(0.020995694443760312+0.019694851744973672j)|1 1 2 2 2>
(0.10389218414949343+0.09745527445003588j)|1 1 2 1 2>
(-0.2039953555962125-0.1913562943051796j)|1 1 2 1 1>
(-0.22902448417443833-0.21483467831278547j)|1 1 1 1 2>
(0.5027731940937056+0.47162257697769666j)|1 1 1 1 1>
(-0.1926675551007114-0.1807303370666639j)|1 1 1 0 1>
(0.08635559498814475+0.08100521014882706j)|1 0 1 1 2>
(-0.19266755510063407-0.18073033706632566j)|1 0 1 1 1>
(0.08197172643895172+0.07689295554458803j)|1 0 1 0 1>
(0.0149496019784